# Importing Modules

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

# General Utilities

## Render Images

### Input: the images & their titles; others configuartions are optional

### Description: Since we'll be previewing alot of image (it's a computer vision course), it's quite handy to have a rendering function

In [ ]:
def RenderImage(
    images: list[np.ndarray], titles: list[str], 
    nrows = 1, ncols = 3, figsize = (36, 48), cmap = 'gray'
    ) -> None:

    plt.subplots(nrows = nrows, ncols = ncols, figsize = figsize)
    for i in range((len(images))):
        plt.subplot(nrows, ncols, (i + 1))
        plt.title(titles[i])
        plt.imshow(images[i], cmap = cmap)

## Transform To Grey Scale

### Input: Image 2D array & Color Weights (optional), by default is set to the weights of the specified rule

### Output: A 2D array representing the greyscale version of the image

### Description: Converting colored data to greyscale according to

$$I_{grey}(p) = 0.3 \times I_{red}(p) + 0.59 \times I_{green}(p) + 0.11 \times I_{blue}(p)$$

In [ ]:
def TransformImage(image: np.ndarray, color_weights = [0.3, 0.59, 0.11]) -> np.ndarray:
    return image.dot(color_weights)

# Part I - Histogram Equalization

## Calculate Histogram

### Input: Image
### Output: Array of histogram counts
### Description: This method is responsible for calculating the normal histogram count at each different grey level with its number of pixels

In [ ]:
def CalculateHistogram(image: np.ndarray) -> np.ndarray:
    pass

## Calculate Cumulative Histogram

### Input: The output array of the method Calculate Histogram
### Output: Array of the cumulative histogram
### Description: In this method, you have to calculate the cumulative histogram by summing up the counts of the pixels ascendingly

In [ ]:
def CalculateCumulativeHistogram(countHistogram: np.ndarray) -> np.ndarray:
    pass

## Calculate Equalized Histogram

### Input: The output array of method Calcuate Cumulative Histogram
### Output: Array of the new pixel intensities that correspond to the original ones
### Description: In this method, you are required to calculate the equalized histogram upon the following equation using the cumulative histogram calculated above

$$T(P) = \frac{255}{(N \times M) - [H_c(0)]} \times [H_c(P) - H_c(0)]$$

where $T(P)$ is the new intensity calculated for intensity $P$, $(N \times M)$ is the size of the image rows multiplied by columns, $H_c(P)$ is the cumulative histogram of the pixel intensity & $H_c(0)$ is the cumulative histogram of pixel intensity 0

In [ ]:
def CalculateEqualizedHistogram(cumulativeHistogram: np.ndarray) -> np.ndarray:
    pass

## Calculate Equalized Image

### Input: Original image & the output array of method Calculate Equalized Histogram
### Output: The equalized image
### Description: In this method, you are required t ogenerate the new equalized image after manipulating the old pixel intensity values by teh new values of intensities obtained after calculating the equalized histogram

In [ ]:
def CalculateEqualizedImage(image: np.ndarray, equalizedHistorgram: np.ndarray) -> np.ndarray:
    pass

# Part II - Optimal Thresholding

## Segment Optimal Thresholding

### Input: Image
### Output: Segmented image
### Description: Implements the iterative optimal thresholding 

In [ ]:
def OptimalThresholding(image: np.ndarray) -> np.ndarray:
    pass

# Part III - Putting Everything together